# Mesh 3D

Testing notebook to mesh in 3D

In [1]:
import pandas as pd
import time
import os
import shutil
import numpy as np
from flow_steady import RunFlow
import geo_to_bc_dict as bc_dict
import subprocess
import yaml
import sys
# import crop_image_dynamic
# import process_images
from mesh import MeshGenerator
import sys
sys.path.append('/home/joshgregory/clotsimnet/data_gen_3d/scripts/sources')
import sources.randSeqAdd as RSA
import sources.gmshUtilities as GMU

In [3]:
boxRSA, success = RSA.getBoxRSA3D(
                seed=seed, 
                a_LowerBounds=boxLower, 
                a_UpperBounds=boxUpper,  
                a_N=num_pores,
                a_R0=radius,
                max_time_sec=max_time_sec
            )

boxRSA in 3D has the following layout:

1st column: xP

2nd column: yP

3rd column: zP

4th column: rP/particle radius

Maximum density for 3d spheres from [here](https://en.wikipedia.org/wiki/Random_sequential_adsorption#Saturation_coverages_for_disks,_spheres,_and_hyperspheres):

0.3841307

0.38278

0.384

Which corresponds to porosities of

0.615869

0.61722

0.616

In [ ]:
# np.savetxt('fixed-Porosity-test.txt', boxRSA)

# GMU.xyzBoxPackingGeoWriterFixed(boxRSA, 'fixed-Porosity-test.txt', 'fixed-Porosity-test.geo', 0.01)

# phi = RSA.getPorosity3D(a_LowerBounds=boxLower, a_UpperBounds=boxUpper, a_PosRSA=boxRSA)

# print(f'Porosity: {phi}')

In [4]:
print(boxRSA)

[[8.34043980e-01 7.20324516e-01 1.14374816e-04 1.70000009e-02]
 [6.04665160e-01 1.46755889e-01 9.23385918e-02 1.70000009e-02]
 [3.72520417e-01 3.45560730e-01 3.96767467e-01 1.70000009e-02]
 ...
 [1.34062064e+00 8.11852932e-01 3.88667476e-03 1.70000009e-02]
 [7.37093836e-02 9.14859235e-01 5.26784593e-03 1.70000009e-02]
 [4.83869374e-01 4.33486462e-01 4.04291961e-04 1.70000009e-02]]


# 3D Search Space

In [5]:
import pandas as pd
import time
import os
import shutil
import numpy as np
from flow_steady import RunFlow
import geo_to_bc_dict as bc_dict
import subprocess
import yaml
import sys
# import crop_image_dynamic
# import process_images
from mesh import MeshGenerator
import sys
sys.path.append('/home/joshgregory/clotsimnet/data_gen_3d/scripts/sources')
import sources.randSeqAdd as RSA
import sources.gmshUtilities as GMU

In [4]:
num_pores = 8000
radius = 0.024
seed = 1
max_time_sec = float(10000)
mesh_size = 0.025 # Mess with this, keep it below 4 million cells

case_dir = f'/home/joshgregory/clotsimnet/data_gen_3d/aN_{num_pores}_rp_024_seed_1'

# Create case directory
os.makedirs(case_dir, exist_ok=True)

print(f'Case dir: {case_dir}')

# Extract simulation id from the given output directory
sim_id = os.path.basename(case_dir)
print(f'Sim id: {sim_id}')

boxLower = np.array([0.0, 0.0, 0.0])
boxUpper = np.array([2.0, 1.0, 1.0])
box = np.array([[0.0, 0.0, 0.0],[2.0, 1.0, 1.0]])

print('Running RSA code')
boxRSA, success = RSA.getBoxRSA3D(
                seed=seed, 
                a_LowerBounds=boxLower, 
                a_UpperBounds=boxUpper,  
                a_N=num_pores,
                a_R0=radius,
                max_time_sec=max_time_sec
            )

phi = RSA.getPorosity3D(a_LowerBounds=boxLower, a_UpperBounds=boxUpper, a_PosRSA=boxRSA)

print(f'Occupied packing fraction: {phi}')
print(f'Unoccupied packing fraction: {1-phi}')

text_path = os.path.join(case_dir, f'{sim_id}.txt')
geo_path = os.path.join(case_dir, f'{sim_id}.geo')

# Save particle positions
np.savetxt(text_path, boxRSA)

# Generate .geo file
print('Generating .geo file')
GMU.xyzBoxPackingGeoWriterFixed(a_Box=box, a_XYZFile=text_path, a_GeoFile=geo_path, a_Sizing=mesh_size)

# Open existing .geo file and read the contents
with open(geo_path, 'r') as f:
    content = f.read()

# Write the Mesh version line to the .geo file while it's still open
with open(geo_path, 'w') as f:
    f.write('Mesh.MshFileVersion = 2.0;\n' + content)

# Change directory and run geo2h5 command for CFD mesh later
original_dir = os.getcwd()
os.chdir(case_dir)
        
command = 'geo2h5'
# command = '/projects/jogr4852/FLATiron/src/flatiron_tk/scripts/geo2h5' # For Alpine
args = ['-m', geo_path, '-d', '3', '-o', 
        os.path.join(case_dir, f'{sim_id}')]

print('Running geo2h5')
result = subprocess.run([command] + args, 
                        capture_output=True, 
                        text=True)

if result.returncode != 0:
    print(f"geo2h5 command failed with error: {result.stderr}")

Case dir: /home/joshgregory/clotsimnet/data_gen_3d/aN_8000_rp_024_seed_1
Sim id: aN_8000_rp_024_seed_1
Running RSA code
Occupied packing fraction: 0.23162334920390498
Unoccupied packing fraction: 0.768376650796095
Generating .geo file
GMSH UTILITIES BULLSHIT
0.0
0.0
0.0
Running geo2h5


Packing fraction of 0.1:

* num_pores = 6000
* radius = 0.02
* Takes about 6 minutes

Packing fraction of about 0.23:

* num_pores = 8000
* radius = 0.024
* Takes about 20 minutes

Anything at higher packing fractions tends to take over 45 minutes and does not converge.

In [2]:
num_pores = 8000
radius = 0.024
seed = 1
max_time_sec = float(10000)
mesh_size = 0.025 # Mess with this, keep it below 4 million cells

case_dir = f'/home/joshgregory/clotsimnet/data_gen_3d/test_geos_fine_2/aN_8000_rp_024_seed_1'

# Create case directory
os.makedirs(case_dir, exist_ok=True)

print(f'Case dir: {case_dir}')

# Extract simulation id from the given output directory
sim_id = os.path.basename(case_dir)
print(f'Sim id: {sim_id}')

boxLower = np.array([0.0, 0.0, 0.0])
boxUpper = np.array([2.0, 1.0, 1.0])
box = np.array([[0.0, 0.0, 0.0],[2.0, 1.0, 1.0]])

# print('Running RSA code')
# boxRSA, success = RSA.getBoxRSA3D(
#                 seed=seed, 
#                 a_LowerBounds=boxLower, 
#                 a_UpperBounds=boxUpper,  
#                 a_N=num_pores,
#                 a_R0=radius,
#                 max_time_sec=max_time_sec
#             )

# phi = RSA.getPorosity3D(a_LowerBounds=boxLower, a_UpperBounds=boxUpper, a_PosRSA=boxRSA)

# print(f'Occupied packing fraction: {phi}')
# print(f'Unoccupied packing fraction: {1-phi}')

text_path = '/home/joshgregory/clotsimnet/data_gen_3d/test_geos_fine_2/aN_8000_rp_024_seed_1/aN_8000_rp_024_seed_1.txt'
geo_path = '/home/joshgregory/clotsimnet/data_gen_3d/test_geos_fine_2/aN_8000_rp_024_seed_1/aN_8000_rp_024_seed_1.geo'

# Save particle positions
# np.savetxt(text_path, boxRSA)

# Generate .geo file
print('Generating .geo file')
GMU.xyzBoxPackingGeoWriterFixed(a_Box=box, a_XYZFile=text_path, a_GeoFile=geo_path, a_Sizing=mesh_size)

# Open existing .geo file and read the contents
with open(geo_path, 'r') as f:
    content = f.read()

# Write the Mesh version line to the .geo file while it's still open
with open(geo_path, 'w') as f:
    f.write('Mesh.MshFileVersion = 2.0;\n' + content)

# Change directory and run geo2h5 command for CFD mesh later
original_dir = os.getcwd()
os.chdir(case_dir)
        
command = 'geo2h5'
# command = '/projects/jogr4852/FLATiron/src/flatiron_tk/scripts/geo2h5' # For Alpine
args = ['-m', geo_path, '-d', '3', '-o', 
        os.path.join(case_dir, f'{sim_id}')]

print('Running geo2h5')
result = subprocess.run([command] + args, 
                        capture_output=True, 
                        text=True)

if result.returncode != 0:
    print(f"geo2h5 command failed with error: {result.stderr}")

Case dir: /home/joshgregory/clotsimnet/data_gen_3d/test_geos_fine_2/aN_8000_rp_024_seed_1
Sim id: aN_8000_rp_024_seed_1
Generating .geo file
GMSH UTILITIES BULLSHIT
0.0
0.0
0.0
Running geo2h5
